In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())
    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/2D Klein Gordon/stan'

In [4]:
# !pip install smt

In [5]:
def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
    y = xt[:,0]*np.cos(xt[:,1])
    return y.reshape(-1,1)    

In [6]:
label = "KG_tanh"
loss_thresh = 0.01

x = np.linspace(-5,5,500).reshape(-1,1)
t = np.linspace(0,10,1000).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

y_true = true_2D_1(xt)
y_true_norm = np.linalg.norm(y_true,2)

#bound_pts_idx = ((X == -5) + (X == 5) + (T == 0)).reshape(-1,)

#xt_bound = xt[bound_pts_idx,:]
#y_bound = y_true[bound_pts_idx,:]


xt_test_tensor = torch.from_numpy(xt).float().to(device)


lb_xt = xt[0]
ub_xt = xt[-1]

In [7]:
def trainingdata(N_I,N_B,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    x_BC1 = np.random.uniform(size = N_I).reshape(-1,1)
    t_BC1 = np.zeros((N_I,1))
    samples = np.hstack((x_BC1,t_BC1))
    xt_BC1 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC1 = true_2D_1(xt_BC1)
    
    x_BC2 = np.zeros((int(N_B/2),1))
    t_BC2 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC2,t_BC2))
    xt_BC2 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC2 = true_2D_1(xt_BC2)
    
    x_BC3 = np.ones((int(N_B/2),1))
    t_BC3 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC3,t_BC3))
    xt_BC3 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC3 = true_2D_1(xt_BC3)

    xt_BC = np.vstack((xt_BC1,xt_BC2,xt_BC3))
    y_BC = np.vstack((y_BC1,y_BC2,y_BC3))

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll, xt_BC)) # append training points to collocation points 

    return xt_coll, xt_BC, y_BC

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
        

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC(self,xt,y):
                
        loss_bc = self.loss_function(self.forward(xt), y)
                
        return loss_bc
    
    def loss_PDE(self, xt_coll, f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        y = self.forward(g) 
        
        
        y_x_t = autograd.grad(y,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        y_xx_tt = autograd.grad(y_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dx = u_x_t[:,[0]]
        
        d2y_dx2 = y_xx_tt[:,[0]]
        d2y_dt2 = y_xx_tt[:,[1]]    
        

        f = d2y_dt2 - d2y_dx2 + torch.pow(y,2) + (g[:,0]*torch.cos(g[:,1])).reshape(-1,1) - (torch.pow(g[:,0],2)*torch.pow(torch.cos(g[:,1]),2)).reshape(-1,1)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_BC,y_BC,xt_coll,f_hat):

        loss_BC = self.loss_BC(xt_BC,y_BC)
        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_val = loss_BC + loss_f
        
        return loss_val
         
    'test neural network'
    def test(self):
        y_pred = self.forward(xt_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()
   
        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re  

In [9]:
def data_update(loss_np):
    train_loss.append(loss_np)
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [10]:
def train_step(xt_BC, y_BC, xt_coll, f_hat,seed):
    # x_coll_np_array = colloc_pts(N_f,seed*123)
    # x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
#     # f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
#     xt_coll, xt_BC, y_BC = trainingdata(N_I,N_B,N_f,seed*11)
#     xt_coll = torch.from_numpy(xt_coll).float().to(device)
#     xt_BC = torch.from_numpy(xt_BC).float().to(device)
#     y_BC = torch.from_numpy(y_BC).float().to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_BC, y_BC, xt_coll,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        return loss

    optimizer.step(closure)

In [11]:
def train_model(max_iter,rep): 
  print(rep) 
  torch.manual_seed(rep*9)
  start_time = time.time() 
  thresh_flag = 0

  xt_coll, xt_BC, y_BC = trainingdata(N_I,N_B,N_f,rep*11)
  xt_coll = torch.from_numpy(xt_coll).float().to(device)
  xt_BC = torch.from_numpy(xt_BC).float().to(device)
  y_BC = torch.from_numpy(y_BC).float().to(device)

  f_hat = torch.zeros(xt_coll.shape[0],1).to(device)

  for i in range(max_iter):
    train_step(xt_BC, y_BC, xt_coll,f_hat,i)
    
    loss_np = PINN.loss(xt_BC, y_BC, xt_coll,f_hat).cpu().detach().numpy()
    if(thresh_flag == 0):
        if(loss_np < loss_thresh):
            time_threshold[rep] = time.time() - start_time
            epoch_threshold[rep] = i+1            
            thresh_flag = 1       
    data_update(loss_np)
    print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])

  elapsed_time[rep] = time.time() - start_time  
  print('Training time: %.2f' % (elapsed_time[rep]))

In [12]:
max_reps = 10 #10
max_iter = 100 #100


train_loss_full = []
test_mse_full = []
test_re_full = []

elapsed_time= np.zeros((max_reps,1))
time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

N_I = 200  #Total number of data points for 'y'
N_B = 400
N_f = 10000 #Total number of collocation points

for reps in range(max_reps):
  print(reps)

  train_loss = []
  test_mse_loss = []
  test_re_loss = []


  torch.manual_seed(reps*36)

  layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

  PINN = Sequentialmodel(layers)

  PINN.to(device)

  'Neural Network Summary'
  print(PINN)

  params = list(PINN.parameters())


  optimizer = torch.optim.LBFGS(PINN.parameters(), lr=1, 
                            max_iter = 20, 
                            max_eval = 30, 
                            tolerance_grad = 1e-8, 
                            tolerance_change = 1e-8, 
                            history_size = 100, 
                            line_search_fn = 'strong_wolfe')




  train_model(max_iter,reps)

  torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
  train_loss_full.append(train_loss)
  test_mse_full.append(test_mse_loss)
  test_re_full.append(test_re_loss)
  #elapsed_time[reps] = time.time() - start_time



  #print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 54.50709 Test MSE 7.609332695097407 Test RE 1.3185036811521091
1 Train Loss 40.185413 Test MSE 7.736740745023647 Test RE 1.329496144770292
2 Train Loss 35.97084 Test MSE 7.139023616143794 Test RE 1.2771075418426558
3 Train Loss 31.473644 Test MSE 6.5440755344896875 Test RE 1.222734629

92 Train Loss 0.0015979385 Test MSE 0.0004174941430326947 Test RE 0.00976637180641998
93 Train Loss 0.0015763225 Test MSE 0.00040381253404449 Test RE 0.009605013017267471
94 Train Loss 0.0013410567 Test MSE 0.0003175262483299416 Test RE 0.008517220217697168
95 Train Loss 0.001235441 Test MSE 0.0002593892785316492 Test RE 0.007698109261720236
96 Train Loss 0.0011941027 Test MSE 0.0002738595246299188 Test RE 0.007909918072051086
97 Train Loss 0.0011426368 Test MSE 0.00023516637325288862 Test RE 0.0073298598534792874
98 Train Loss 0.0010992301 Test MSE 0.00021634441751671365 Test RE 0.0070304141108504005
99 Train Loss 0.0010488344 Test MSE 0.00016867117590566106 Test RE 0.00620766821129004
Training time: 127.85
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3)

85 Train Loss 0.042155374 Test MSE 0.04739139609831875 Test RE 0.10405376802810426
86 Train Loss 0.041106652 Test MSE 0.04577373916206272 Test RE 0.10226246464623412
87 Train Loss 0.04054768 Test MSE 0.04570031627405776 Test RE 0.1021804152114661
88 Train Loss 0.038947523 Test MSE 0.04373301378714897 Test RE 0.09995689656505083
89 Train Loss 0.038022317 Test MSE 0.041745083647706425 Test RE 0.09765865261717928
90 Train Loss 0.036985826 Test MSE 0.04073221997521305 Test RE 0.09646662856720431
91 Train Loss 0.03560885 Test MSE 0.04036102547676129 Test RE 0.09602607026497965
92 Train Loss 0.03459997 Test MSE 0.038801290964679755 Test RE 0.09415234645762897
93 Train Loss 0.03375441 Test MSE 0.03632242868765491 Test RE 0.09109520136252879
94 Train Loss 0.032951128 Test MSE 0.03752047885327933 Test RE 0.09258534441708902
95 Train Loss 0.031868204 Test MSE 0.040358924930563894 Test RE 0.0960235714456504
96 Train Loss 0.030817281 Test MSE 0.04056310608922701 Test RE 0.09626616300430434
97 Trai

78 Train Loss 0.0028150484 Test MSE 0.000819017092107544 Test RE 0.013679006088777458
79 Train Loss 0.002510628 Test MSE 0.0006607928318258681 Test RE 0.012286854682437037
80 Train Loss 0.002364831 Test MSE 0.000606521006392982 Test RE 0.01177147768683951
81 Train Loss 0.0021757642 Test MSE 0.00045230638162132886 Test RE 0.010165398723960129
82 Train Loss 0.0019014223 Test MSE 0.00033655397093928634 Test RE 0.008768704196290555
83 Train Loss 0.0016986188 Test MSE 0.0002537709710563543 Test RE 0.0076142832952174905
84 Train Loss 0.0015912617 Test MSE 0.0002761090570300126 Test RE 0.007942338391021792
85 Train Loss 0.0014751086 Test MSE 0.00027786837794648706 Test RE 0.00796760183988958
86 Train Loss 0.0013885095 Test MSE 0.00026612218846820316 Test RE 0.007797378269446774
87 Train Loss 0.0013000617 Test MSE 0.0002316103714618437 Test RE 0.007274230555577318
88 Train Loss 0.0012505772 Test MSE 0.00020527177450758825 Test RE 0.006848140762407854
89 Train Loss 0.0011828956 Test MSE 0.00018

72 Train Loss 0.7476583 Test MSE 6.389277008526829 Test RE 1.208186333927612
73 Train Loss 0.7408482 Test MSE 6.420383267486442 Test RE 1.2111237966216335
74 Train Loss 0.73336136 Test MSE 6.446993650607469 Test RE 1.2136310568435638
75 Train Loss 0.7258501 Test MSE 6.469298654162965 Test RE 1.2157286757497028
76 Train Loss 0.7195274 Test MSE 6.491800713220304 Test RE 1.217841165284524
77 Train Loss 0.7115152 Test MSE 6.505366848160058 Test RE 1.2191129830735523
78 Train Loss 0.7042143 Test MSE 6.537504528216606 Test RE 1.2221205916125408
79 Train Loss 0.6974133 Test MSE 6.567148947650057 Test RE 1.2248883203234833
80 Train Loss 0.6909097 Test MSE 6.567712354540854 Test RE 1.2249408618239745
81 Train Loss 0.68701273 Test MSE 6.5983163621425645 Test RE 1.2277915141843265
82 Train Loss 0.6818762 Test MSE 6.614068083104452 Test RE 1.2292561530296726
83 Train Loss 0.6764307 Test MSE 6.632301824347487 Test RE 1.2309494008038186
84 Train Loss 0.668904 Test MSE 6.662609538016289 Test RE 1.233

...

59 Train Loss 18.071045 Test MSE 8.054206975359904 Test RE 1.3564989227040263
60 Train Loss 18.006058 Test MSE 8.063652772861953 Test RE 1.357294125733443
61 Train Loss 17.931988 Test MSE 8.019507548701059 Test RE 1.3535737097431368
62 Train Loss 17.76373 Test MSE 8.199159393804601 Test RE 1.3686510184750702
63 Train Loss 17.62661 Test MSE 8.110086681238935 Test RE 1.3611964515640465
64 Train Loss 17.527592 Test MSE 8.104475969071334 Test RE 1.3607255193067989
65 Train Loss 17.447535 Test MSE 8.15144421703138 Test RE 1.364662761192632
66 Train Loss 17.295033 Test MSE 8.20240639113165 Test RE 1.3689219954190515
67 Train Loss 17.049099 Test MSE 8.093097477469314 Test RE 1.3597699706560404
68 Train Loss 16.897264 Test MSE 8.078623619050049 Test RE 1.3585535065335463
69 Train Loss 16.806786 Test MSE 8.190161104582508 Test RE 1.3678997891152034
70 Train Loss 16.62933 Test MSE 8.095529631837236 Test RE 1.3599742757475466
71 Train Loss 16.55249 Test MSE 7.929699279635379 Test RE 1.34597322021

55 Train Loss 0.079468235 Test MSE 0.018812632283405063 Test RE 0.06555907897774321
56 Train Loss 0.073313385 Test MSE 0.016198283182923854 Test RE 0.06083346497270699
57 Train Loss 0.07017259 Test MSE 0.01610343239677086 Test RE 0.06065509503077093
58 Train Loss 0.066218 Test MSE 0.01647082897318903 Test RE 0.061343109838257054
59 Train Loss 0.05971183 Test MSE 0.014779839172760743 Test RE 0.058108934800906856
60 Train Loss 0.056732673 Test MSE 0.015494552725305458 Test RE 0.059497344463911625
61 Train Loss 0.052890297 Test MSE 0.014084490701943016 Test RE 0.05672553936543616
62 Train Loss 0.05086296 Test MSE 0.014815458128562286 Test RE 0.05817891303376672
63 Train Loss 0.04949251 Test MSE 0.013849782257789225 Test RE 0.0562509074659498
64 Train Loss 0.04736457 Test MSE 0.013555464102639777 Test RE 0.0556500111778291
65 Train Loss 0.044248495 Test MSE 0.013132623256172883 Test RE 0.05477517869651174
66 Train Loss 0.040364586 Test MSE 0.012218619516993665 Test RE 0.05283468602209797
6

48 Train Loss 7.7276487 Test MSE 2.0047292136390116 Test RE 0.6767619293139832
49 Train Loss 7.679315 Test MSE 1.9669971993772517 Test RE 0.6703628382649472
50 Train Loss 7.635895 Test MSE 1.9445025722145268 Test RE 0.6665186734211519
51 Train Loss 7.5448275 Test MSE 2.078326839195215 Test RE 0.6890726034251935
52 Train Loss 7.4925117 Test MSE 2.0700403485287326 Test RE 0.68769753177177
53 Train Loss 7.43316 Test MSE 2.0618612376535426 Test RE 0.6863375772474551
54 Train Loss 7.3110743 Test MSE 2.1145663844314226 Test RE 0.6950542803909581
55 Train Loss 7.1725626 Test MSE 2.1539654260713714 Test RE 0.7014995944566972
56 Train Loss 7.111928 Test MSE 2.1416534291278264 Test RE 0.6994918467550627
57 Train Loss 7.027411 Test MSE 2.1174551242956645 Test RE 0.6955288802329329
58 Train Loss 6.9680195 Test MSE 2.104819039071021 Test RE 0.6934504623036886
59 Train Loss 6.8253613 Test MSE 2.1625049466880806 Test RE 0.7028887868100208
60 Train Loss 6.7062674 Test MSE 2.2182903823445534 Test RE 0.

44 Train Loss 2.589126 Test MSE 2.1653918299944546 Test RE 0.7033577987173578
45 Train Loss 2.5859065 Test MSE 2.161972183782846 Test RE 0.7028021983054059
46 Train Loss 2.5820317 Test MSE 2.1543475072131604 Test RE 0.7015618094481055
47 Train Loss 2.5781095 Test MSE 2.159684488037091 Test RE 0.7024302640224221
48 Train Loss 2.5769565 Test MSE 2.1616488856992953 Test RE 0.7027496483465079
49 Train Loss 2.5729578 Test MSE 2.1548401256866345 Test RE 0.7016420152997963
50 Train Loss 2.5683174 Test MSE 2.1602012359035423 Test RE 0.702514294266735
51 Train Loss 2.5657701 Test MSE 2.1664512295803027 Test RE 0.7035298335968306
52 Train Loss 2.5633502 Test MSE 2.1666382287865082 Test RE 0.7035601958500471
53 Train Loss 2.5609574 Test MSE 2.167790769570591 Test RE 0.7037473000007896
54 Train Loss 2.5593624 Test MSE 2.16143243466431 Test RE 0.702714463470341
55 Train Loss 2.5551114 Test MSE 2.163257950960398 Test RE 0.7030111523549786
56 Train Loss 2.551022 Test MSE 2.163080755474465 Test RE 0.7

39 Train Loss 2.3764653 Test MSE 2.1103893442398425 Test RE 0.6943674481264351
40 Train Loss 2.0182567 Test MSE 1.8891766323017565 Test RE 0.6569681915215658
41 Train Loss 1.5848811 Test MSE 1.724910569402612 Test RE 0.6277566932743719
42 Train Loss 1.1408852 Test MSE 1.5535000915206776 Test RE 0.5957495210476513
43 Train Loss 0.7606033 Test MSE 1.0326286452549875 Test RE 0.485713466358395
44 Train Loss 0.48958686 Test MSE 0.5150331986973863 Test RE 0.34302492055129163
45 Train Loss 0.361996 Test MSE 0.24112884608966592 Test RE 0.23471056787580696
46 Train Loss 0.30300757 Test MSE 0.20538652206475372 Test RE 0.21661774508961287
47 Train Loss 0.23434614 Test MSE 0.1864868785529115 Test RE 0.20641069689769143
48 Train Loss 0.20679659 Test MSE 0.1549544677418826 Test RE 0.18815255128386138
49 Train Loss 0.17184299 Test MSE 0.127322070158258 Test RE 0.17055321322267925
50 Train Loss 0.14675908 Test MSE 0.11134059548807067 Test RE 0.1594905038903923
51 Train Loss 0.12724647 Test MSE 0.07462

In [13]:
import scipy.io as sio

In [14]:
for tune_reps in range(5):
    label = "KG_tanh_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(np.mean(re[:,-1]))

FileNotFoundError: [Errno 2] No such file or directory: 'KG_tanh_tune0.mat'

In [15]:
a = 0
for i in range(10):
    a = a+ test_re_full[i][-1]
    print(test_re_full[i][-1])
    
print("a = ",a/10)

0.00620766821129004
0.09207865958258454
0.0039923813402715945
1.2569464859652497
0.01438868208707847
1.350745778552734
0.030925378830068205
0.7004777223704535
0.03525212872970865
0.03774196068387512
a =  0.35287568463533137
